In [2]:
import numpy as np

In [1]:
n_input_bits = 255
n_output_bits = 8
n_point_bits = 32
n_points = 1000
n_cluster_create_bits = 6
n_cluster_activate_bits = 4

In [3]:
def gen_input_output(count=1000, n_in_active_bits=40, n_out_active_bits=4) -> (np.array, np.array):
    inp = np.zeros(shape=(count, n_input_bits), dtype=np.int8)
    out = np.zeros(shape=(count, n_output_bits), dtype=np.int8)
    for i in range(count):
        in_bits = np.random.choice(n_input_bits, n_in_active_bits, replace=False)
        out_bits = np.random.choice(n_output_bits, n_out_active_bits, replace=False)
        inp[i][in_bits] = 1
        out[i][out_bits] = 1
    return inp, out

def gen_points() -> (np.array, np.array):
    point_ins = np.zeros(shape=(n_points, n_point_bits), dtype=np.int32)
    point_outs = np.zeros(n_points, dtype=np.int32)
    for i in range(n_points):
        point_ins[i] = np.random.choice(n_input_bits, n_point_bits, replace=False)
        point_outs[i] = np.random.choice(n_output_bits, 1)
    return point_ins, point_outs   
                

In [10]:
in_data, out_data = gen_input_output(count=1000, n_in_active_bits=50)

# print(in_data)
# print(out_data)

point_in_bits, point_out_bits = gen_points()

# print(point_in_bits)
# print(point_out_bits)

point_clusters = [np.empty(shape=(0, n_point_bits), dtype=np.int8) for _ in range(n_points)]
cluster_stats = [[] for _ in range(n_points)]

In [75]:
# filter active points for input bit vector
point_masks = in_data[0][point_in_bits]
len(np.where(np.sum(point_masks, axis=1) > n_cluster_activate_bits)[0])

618

In [11]:
# test visualisation

print(in_data[0][point_in_bits])
print(in_data[0])
print(point_in_bits[0])
print(np.where(in_data[0] > 0))
print(set(point_in_bits[0]) & set(np.where(in_data[0] > 0)[0]))
print(in_data[0][point_in_bits][0])

[[0 0 0 ..., 1 0 0]
 [0 0 0 ..., 1 1 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [0 1 0 ..., 0 1 0]
 [1 0 1 ..., 0 0 0]]
[0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0]
[122   4  56   2 247 103 113 123   3  77 241 208  29 200 224 121  64 198
 205 118  65  94  87  51  27 144 249 156 187 225 116 244]
(array([  1,   7,   9,  15,  16,  17,  32,  44,  45,  47,  62,  74,  76,
        77,  80,  88,  91,  99, 106, 110, 113, 115, 118, 124, 128, 131,
       135, 141, 145, 147, 154, 169, 175, 178, 183, 185, 195, 199, 200,
      

In [12]:
from numba import njit

def clear_clusters():
    global point_clusters, cluster_stats
    point_clusters = [np.empty(shape=(0, n_point_bits), dtype=np.int8) for _ in range(n_points)]
    cluster_stats = [[] for _ in range(n_points)] # [[{} {} ...] ...]

def cluster_count():
    global point_clusters
    return sum(len(clusters) for clusters in point_clusters)

def append_cluster(point_idx: int, mask: np.array):
    global point_clusters
    clusters = point_clusters[point_idx]
    isects = np.count_nonzero(clusters & mask, axis=1) # bit-intersection counts
    if len(isects) == 0 or np.max(isects) < np.count_nonzero(mask):
#         print('append ', mask)
        point_clusters[point_idx] = np.vstack((clusters, mask))
    
def update_clusters(point_idx: int, mask: np.array):
    pass
    
def process_input(input_bits: np.array):
    # filter active points 
    point_masks = input_bits[point_in_bits]
    point_sums = np.sum(point_masks, axis=1)
    for point_idx in range(n_points):
        if point_sums[point_idx] >= n_cluster_create_bits:
            append_cluster(point_idx, point_masks[point_idx])
        update_clusters(point_idx, point_masks[point_idx])
    


In [13]:
%%time

# feed data
clear_clusters()

for bit_vector in in_data:
    process_input(bit_vector)

Wall time: 33.6 s


In [9]:
cluster_count()

4945344